In [ ]:
!pip install flask pandas scikit-learn vaderSentiment nltk requests aiohttp

In [ ]:
import requests

In [ ]:
API_KEY = 'your_newsapi_key'

In [ ]:
def fetch_news(country):
    url = f"https://newsapi.org/v2/top-headlines?country={country}&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        return None

In [ ]:
# Example usage
if __name__ == "__main__":
    articles = fetch_news('us')
    for article in articles:
        print(article['title'])

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]  
    tokens = [word.lower() for word in tokens] 
    tokens = [word for word in tokens if word not in stop_words]  
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [ ]:
# Example usage
if __name__ == "__main__":
    text = "The economy is doing well. Stock markets are up!"
    print(preprocess_text(text))

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [ ]:
# Example usage
if __name__ == "__main__":
    text = "The economy is doing well. Stock markets are up!"
    print(get_sentiment(text))

In [ ]:
import pandas as pd
from sentiment_analysis import get_sentiment, analyzer

In [ ]:
financial_data = {
    'GDP_growth_rate': {
        'us': 2.3,
        'uk': 1.8,
        'de': 1.5,
        'jp': 0.7,
        'cn': 6.0,
        'in': 5.5,
        'fr': 1.7,
        'it': 0.3,
        'ca': 1.9,
        'au': 2.2
    },
    'inflation_rate': {
        'us': 1.5,
        'uk': 2.1,
        'de': 1.4,
        'jp': 0.3,
        'cn': 2.8,
        'in': 3.8,
        'fr': 1.3,
        'it': 0.5,
        'ca': 1.7,
        'au': 1.6
    }
}

In [ ]:
def extract_features(text_data, country_data):
    features = []
    for text, country in text_data:
        sentiment = get_sentiment(text)
        sentiment_score = analyzer.polarity_scores(text)['compound']
        
        gdp_growth = country_data['GDP_growth_rate'].get(country, 0.0)
        inflation = country_data['inflation_rate'].get(country, 0.0)
        
        features.append({
            'sentiment': sentiment,
            'sentiment_score': sentiment_score,
            'GDP_growth_rate': gdp_growth,
            'inflation_rate': inflation
        })
    return pd.DataFrame(features)


In [ ]:
# Example usage
if __name__ == "__main__":
    text_data = [
        ("The economy is doing well.", 'us'),
        ("Stock markets are down.", 'uk')
    ]
    features = extract_features(text_data, financial_data)
    print(features)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
data = pd.DataFrame({
    'sentiment_score': [0.5, -0.3, 0.1, -0.8, 0.2],
    'target': [0.01, -0.02, 0.005, -0.03, 0.008]
})

In [ ]:
X = data.drop('target', axis=1)
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)

In [ ]:
def generate_recommendations(predictions, threshold=0.01):
    recommendations = []
    for i, pred in enumerate(predictions):
        if pred > threshold:
            recommendations.append(f"Buy currency pair {i}")
        elif pred < -threshold:
            recommendations.append(f"Sell currency pair {i}")
        else:
            recommendations.append(f"Hold currency pair {i}")
    return recommendations

In [ ]:
if __name__ == "__main__":
    predictions = [0.02, -0.01, 0.005, -0.03, 0.008]
    recommendations = generate_recommendations(predictions)
    for rec in recommendations:
        print(rec)